# UR5e Touch-Hold-Retreat Motion

This notebook demonstrates a touch-and-retreat motion using the UR5e robot with NetFT probe.

Instead of pick-and-place with a gripper (like the original IIWA example), we:
1. Move TCP to touch brick A
2. Hold contact briefly (1 second)
3. Retreat
4. Move TCP to touch brick B
5. Hold contact briefly (1 second)
6. Retreat to home position

This demonstrates the same trajectory generation and Jacobian-based velocity control concepts, adapted for the UR5e's 6-DOF kinematics and the TCP probe end-effector.

In [ ]:
import sys
from pathlib import Path

# Add parent directory to path for ur5e_description imports
sys.path.insert(0, str(Path.cwd().parent))

import numpy as np
from pydrake.all import (
    DiagramBuilder,
    Integrator,
    JacobianWrtVariable,
    LeafSystem,
    PiecewisePose,
    Rgba,
    RigidTransform,
    RotationMatrix,
    Simulator,
    StartMeshcat,
    TrajectorySource,
)

from manipulation import running_as_notebook
from ur5e_description.ur5e_station import (
    MakeUR5eHardwareStationWithScene,
    GetPlantFromStation,
)

In [ ]:
# Start the visualizer
meshcat = StartMeshcat()

# TCP Contact Frames

For touch-hold-retreat, we define keyframe poses for the TCP (Tool Center Point):

- **pretouch**: Above the brick (approach position)
- **touch**: TCP tip contacts brick surface
- **posttouch**: Same as pretouch (retreat position)

The TCP approaches from above with Z-axis pointing down toward the brick.

```
    TCP (probe tip)
         |
         v  (Z axis pointing down)
    +---------+
    |  Brick  |  <- Top surface
    +---------+
```

In [ ]:
def MakeTCPFrames(X_TCP_initial, X_BrickA, X_BrickB):
    """
    Creates keyframe poses for touch-hold-retreat motion on two bricks.
    
    Args:
        X_TCP_initial: Initial TCP pose in world frame
        X_BrickA: Pose of brick A base in world frame
        X_BrickB: Pose of brick B base in world frame
    
    Returns:
        X_TCP: dict of keyframe poses
        times: dict of keyframe times
    """
    X_TCP = {"initial": X_TCP_initial}
    
    # TCP orientation: Z-axis pointing down (toward brick)
    # UR5e TCP frame has Z pointing out of probe tip, so flip it
    R_touch = RotationMatrix.MakeXRotation(np.pi)
    
    # Geometry constants
    # NOTE: Foam brick SDF has origin at CENTER, not base!
    # So when welded at Z=0, top surface is at Z = brick_height/2
    brick_height = 0.05           # Foam brick is ~5cm tall
    brick_half_height = brick_height / 2.0  # Origin is at center
    approach_height = 0.08        # 8cm above brick top surface
    contact_offset = 0.005        # 5mm into brick surface (for visible contact)
    
    # Brick A contact frames
    p_BrickA = X_BrickA.translation()
    # Top surface is at Z = p_BrickA[2] + brick_half_height
    top_A = p_BrickA[2] + brick_half_height
    
    X_TCP["pretouch_A"] = RigidTransform(
        R_touch, 
        [p_BrickA[0], p_BrickA[1], top_A + approach_height]
    )
    X_TCP["touch_A"] = RigidTransform(
        R_touch,
        [p_BrickA[0], p_BrickA[1], top_A - contact_offset]
    )
    X_TCP["posttouch_A"] = X_TCP["pretouch_A"]  # Retreat to approach pose
    
    # Clearance pose (midway, elevated for safe transition)
    p_mid = (p_BrickA + X_BrickB.translation()) / 2
    X_TCP["clearance"] = RigidTransform(
        R_touch,
        [p_mid[0], p_mid[1], top_A + 0.15]  # 15cm above brick top
    )
    
    # Brick B contact frames
    p_BrickB = X_BrickB.translation()
    top_B = p_BrickB[2] + brick_half_height
    
    X_TCP["pretouch_B"] = RigidTransform(
        R_touch,
        [p_BrickB[0], p_BrickB[1], top_B + approach_height]
    )
    X_TCP["touch_B"] = RigidTransform(
        R_touch,
        [p_BrickB[0], p_BrickB[1], top_B - contact_offset]
    )
    X_TCP["posttouch_B"] = X_TCP["pretouch_B"]
    
    # Final pose: return to initial
    X_TCP["final"] = X_TCP_initial
    
    # Timing (speed factor controls overall speed)
    speed_factor = 10.0  # seconds per meter of translation
    times = {"initial": 0.0}
    
    # initial -> pretouch_A
    dist = np.linalg.norm(
        X_TCP["pretouch_A"].translation() - X_TCP["initial"].translation()
    )
    times["pretouch_A"] = times["initial"] + speed_factor * dist
    
    # pretouch_A -> touch_A (slow approach)
    times["touch_A_start"] = times["pretouch_A"] + 1.0
    times["touch_A_end"] = times["touch_A_start"] + 1.0  # Hold 1 second
    X_TCP["touch_A_start"] = X_TCP["touch_A"]
    X_TCP["touch_A_end"] = X_TCP["touch_A"]
    
    # touch_A -> posttouch_A
    times["posttouch_A"] = times["touch_A_end"] + 1.0
    
    # posttouch_A -> clearance
    dist = np.linalg.norm(
        X_TCP["clearance"].translation() - X_TCP["posttouch_A"].translation()
    )
    times["clearance"] = times["posttouch_A"] + speed_factor * dist
    
    # clearance -> pretouch_B
    dist = np.linalg.norm(
        X_TCP["pretouch_B"].translation() - X_TCP["clearance"].translation()
    )
    times["pretouch_B"] = times["clearance"] + speed_factor * dist
    
    # pretouch_B -> touch_B (slow approach)
    times["touch_B_start"] = times["pretouch_B"] + 1.0
    times["touch_B_end"] = times["touch_B_start"] + 1.0  # Hold 1 second
    X_TCP["touch_B_start"] = X_TCP["touch_B"]
    X_TCP["touch_B_end"] = X_TCP["touch_B"]
    
    # touch_B -> posttouch_B
    times["posttouch_B"] = times["touch_B_end"] + 1.0
    
    # posttouch_B -> final
    dist = np.linalg.norm(
        X_TCP["final"].translation() - X_TCP["posttouch_B"].translation()
    )
    times["final"] = times["posttouch_B"] + speed_factor * dist
    
    return X_TCP, times

In [ ]:
def MakeTCPPoseTrajectory(X_TCP, times):
    """
    Constructs PiecewisePose trajectory from keyframes.
    Uses linear interpolation for positions and quaternion slerp for orientations.
    """
    keyframe_order = [
        "initial",
        "pretouch_A",
        "touch_A_start",
        "touch_A_end",
        "posttouch_A",
        "clearance",
        "pretouch_B",
        "touch_B_start",
        "touch_B_end",
        "posttouch_B",
        "final",
    ]
    
    sample_times = [times[name] for name in keyframe_order]
    poses = [X_TCP[name] for name in keyframe_order]
    
    return PiecewisePose.MakeLinear(sample_times, poses)

# Pseudo-Inverse Controller for UR5e (6 DOF)

The key difference from the IIWA version:
- UR5e has 6 joints, so the Jacobian is 6x6 (square matrix)
- No need to slice the Jacobian to extract specific joint columns
- The TCP frame is from the `netft_probe` model instance

In [ ]:
class PseudoInverseController(LeafSystem):
    """
    Computes joint velocities from desired TCP spatial velocity using Jacobian pseudoinverse.
    
    For UR5e (6 DOF), the Jacobian is 6x6. When not at a singularity,
    pinv(J) ~ inv(J), giving exact velocity tracking.
    """
    def __init__(self, plant):
        LeafSystem.__init__(self)
        self._plant = plant
        self._plant_context = plant.CreateDefaultContext()
        self._ur5e = plant.GetModelInstanceByName("ur5e")
        
        # TCP frame from NetFT probe end-effector
        self._TCP = plant.GetBodyByName(
            "tcp", plant.GetModelInstanceByName("netft_probe")
        ).body_frame()
        self._W = plant.world_frame()

        # Input ports
        self.V_TCP_port = self.DeclareVectorInputPort("V_W_TCP", 6)
        self.q_port = self.DeclareVectorInputPort("ur5e.position", 6)
        
        # Output port
        self.DeclareVectorOutputPort("ur5e.velocity", 6, self.CalcOutput)

    def CalcOutput(self, context, output):
        V_TCP = self.V_TCP_port.Eval(context)
        q = self.q_port.Eval(context)
        
        self._plant.SetPositions(self._plant_context, self._ur5e, q)
        
        # 6x6 Jacobian - SQUARE! No slicing needed (unlike 7-DOF IIWA)
        J_TCP = self._plant.CalcJacobianSpatialVelocity(
            self._plant_context,
            JacobianWrtVariable.kQDot,
            self._TCP,
            [0, 0, 0],  # Point at TCP origin
            self._W,
            self._W,
        )
        
        # qdot = J^+ @ V_TCP
        v = np.linalg.pinv(J_TCP).dot(V_TCP)
        output.SetFromVector(v)

# Define Scene: Brick Positions

We place two foam bricks in the UR5e workspace:
- Brick A: to the right of the robot
- Brick B: to the left of the robot

In [ ]:
# Define brick positions in UR5e workspace
X_BrickA = RigidTransform(
    RotationMatrix.MakeZRotation(0),
    [0.4, -0.2, 0.0]  # 40cm forward, 20cm right of robot base
)
X_BrickB = RigidTransform(
    RotationMatrix.MakeZRotation(0),
    [0.4, 0.2, 0.0]   # 40cm forward, 20cm left of robot base
)

# Scene models to add to the station
scene_models = [
    ("package://manipulation/hydro/061_foam_brick.sdf", X_BrickA),
    ("package://manipulation/hydro/061_foam_brick.sdf", X_BrickB),
]

print(f"Brick A position: {X_BrickA.translation()}")
print(f"Brick B position: {X_BrickB.translation()}")

# Build Station and Get Initial TCP Pose

In [ ]:
# Create station with bricks (simulation mode)
# For real hardware, set robot_ip="192.168.1.102" (or your robot's IP)
# Note: We use position control since the integrator outputs joint positions
station_for_init = MakeUR5eHardwareStationWithScene(
    scene_models=scene_models,
    robot_ip=None,  # Simulation mode
    meshcat=meshcat,
    control_mode="position"  # Position control for trajectory tracking
)
plant_for_init = GetPlantFromStation(station_for_init)

# Get initial TCP pose from default robot configuration
temp_context = station_for_init.CreateDefaultContext()
temp_plant_context = plant_for_init.GetMyContextFromRoot(temp_context)
tcp_body = plant_for_init.GetBodyByName(
    "tcp", plant_for_init.GetModelInstanceByName("netft_probe")
)
X_TCP_initial = plant_for_init.EvalBodyPoseInWorld(temp_plant_context, tcp_body)

print(f"Initial TCP position: {X_TCP_initial.translation()}")
print(f"Initial TCP rotation (RPY deg): {X_TCP_initial.rotation().ToRollPitchYaw().vector() * 180/np.pi}")

# Generate Keyframes and Trajectory

In [ ]:
# Generate keyframes
X_TCP, times = MakeTCPFrames(X_TCP_initial, X_BrickA, X_BrickB)
print(f"Total motion duration: {times['final']:.2f} seconds")
print("\nKeyframe times:")
for name, t in sorted(times.items(), key=lambda x: x[1]):
    print(f"  {name}: {t:.2f}s")

In [ ]:
# Create trajectory and visualize path in meshcat
traj_X_TCP = MakeTCPPoseTrajectory(X_TCP, times)
traj_p_TCP = traj_X_TCP.get_position_trajectory()
p_TCP = traj_p_TCP.vector_values(traj_p_TCP.get_segment_times())

# Draw the TCP path in orange
meshcat.SetLine("tcp_path", p_TCP, 2.0, rgba=Rgba(1, 0.65, 0))

# Build Complete Control Diagram

The control architecture (adapted from original IIWA example):
```
TrajectorySource(V_TCP) --> PseudoInverseController --> v (joint vel)
                                    ^                      |
station.ur5e.position_measured -----+                      |
                                                           |
                         +-------------+-------------------+
                         |             |
                         v             v
                    Integrator    station.ur5e.velocity_ff
                         |
                         v
                 station.ur5e.position
```

Key difference from basic position control: We pass the joint velocity as feedforward
to the station's controller. This prevents the damping term (kd) from fighting motion.

In [ ]:
# Build the full control diagram
builder = DiagramBuilder()

# Add the station (fresh instance for the simulation)
# Use position control mode - integrator outputs joint positions
station = builder.AddSystem(MakeUR5eHardwareStationWithScene(
    scene_models=scene_models,
    robot_ip=None,  # Simulation mode
    meshcat=meshcat,
    control_mode="position"  # Position control for trajectory tracking
))
station.set_name("station")
plant = GetPlantFromStation(station)

# Create spatial velocity trajectory (derivative of pose trajectory)
traj_V_TCP = traj_X_TCP.MakeDerivative()

# Trajectory source for spatial velocity
V_TCP_source = builder.AddSystem(TrajectorySource(traj_V_TCP))
V_TCP_source.set_name("V_TCP_source")

# Pseudoinverse controller: V_TCP -> joint velocities
controller = builder.AddSystem(PseudoInverseController(plant))
controller.set_name("PseudoInverseController")

# Connect velocity trajectory to controller
builder.Connect(V_TCP_source.get_output_port(), controller.GetInputPort("V_W_TCP"))

# Integrator: joint velocity -> joint position
integrator = builder.AddSystem(Integrator(6))
integrator.set_name("integrator")

# Wire up the system
builder.Connect(controller.get_output_port(), integrator.get_input_port())
# Connect integrator output (positions) to position input port
builder.Connect(integrator.get_output_port(), station.GetInputPort("ur5e.position"))
# Connect controller output (velocities) to velocity feedforward port
# This provides feedforward velocity to avoid damping fighting motion
builder.Connect(controller.get_output_port(), station.GetInputPort("ur5e.velocity_ff"))
builder.Connect(
    station.GetOutputPort("ur5e.position_measured"),
    controller.GetInputPort("ur5e.position")
)

diagram = builder.Build()
diagram.set_name("ur5e_touch_motion")

print("Diagram built successfully!")

# Run Simulation

In [ ]:
# Create simulator
simulator = Simulator(diagram)
context = simulator.get_mutable_context()

# Initialize integrator with current joint positions
q0 = plant.GetPositions(
    plant.GetMyContextFromRoot(context),
    plant.GetModelInstanceByName("ur5e")
)
integrator.set_integral_value(
    integrator.GetMyContextFromRoot(context),
    q0
)

# Force initial visualization
diagram.ForcedPublish(context)

# Run simulation with recording
print(f"Running simulation for {times['final']:.2f} seconds...")
meshcat.StartRecording(set_visualizations_while_recording=False)
simulator.AdvanceTo(times["final"] if running_as_notebook else 0.1)
meshcat.PublishRecording()

print("Simulation complete! Check meshcat for the recorded animation.")

# Hardware Mode (Optional)

To run on real UR5e hardware:
1. Ensure `ur_rtde` is installed: `pip install ur-rtde`
2. Set your robot's IP address
3. Ensure workspace is clear and e-stop is accessible!

**CAUTION**: The real robot will move! Ensure safety protocols are in place.

In [ ]:
# HARDWARE MODE - Uncomment to run on real robot
# WARNING: Robot will move! Ensure workspace is clear!

# ROBOT_IP = "192.168.1.102"  # Change to your robot's IP

# builder_hw = DiagramBuilder()
# station_hw = builder_hw.AddSystem(MakeUR5eHardwareStationWithScene(
#     scene_models=None,  # Real world has its own scene
#     robot_ip=ROBOT_IP,
#     meshcat=meshcat,
#     control_mode="position"  # Position control for trajectory tracking
# ))
# plant_hw = GetPlantFromStation(station_hw)

# # ... same wiring as simulation ...
# # Note: You may want to regenerate trajectories based on actual robot position

print("Hardware mode code is commented out for safety.")
print("Uncomment and modify the ROBOT_IP to run on real hardware.")